In [55]:
import pandas as pd
from typing import Tuple, List, Dict
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [56]:
notspam = np.arange(0, 193)
spam = np.arange(0, 18)

df_list = []

for ns in notspam : 
    df_tmp1 = pd.read_csv('TrainData/notspam/{}_notspam.txt'.format(ns))
    df_tmp1.drop(df_tmp1.columns[[0]], axis = 1, inplace = True)
    f = open('TrainData/notspam/{}_notspam.txt'.format(ns), "r")
    df_tmp1['message'] = f.read()
    df_tmp1['Label'] = 'notspam'
    df_list.append(df_tmp1)

for s in spam : 
    df_tmp2 = pd.read_csv('TrainData/spam/{}_spam.txt'.format(s))
    df_tmp2.drop(df_tmp2.columns[[0]], axis = 1, inplace = True)
    f2 = open('TrainData/spam/{}_spam.txt'.format(s), "r")
    df_tmp2['message'] = f2.read()
    df_tmp2['Label'] = 'spam'
    df_list.append(df_tmp2)


In [57]:
df_list[0]

message  \
> date : sun   15 dec 91 02 : 25 : 02 est > : michael < mmors...  > much related : might consider construction f...  > level   indistinguishable "     here ? " . ' john mcnamara name ' is tautologo...  ' john mcnamara name ' false . tautology   . ( reduplication   Subject: re : 2 . 882 s - > np np\n\n> date : ...   

                                                                                                                                                                                                                                                                           Label  
> date : sun   15 dec 91 02 : 25 : 02 est > : michael < mmors...  > much related : might consider construction f...  > level   indistinguishable "     here ? " . ' john mcnamara name ' is tautologo...  ' john mcnamara name ' false . tautology   . ( reduplication   notspam

In [58]:
# df.drop(df.columns[[0]], axis = 1, inplace = True)
df = pd.concat(df_list).reset_index(drop = True)
# df.drop(df.columns[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]], axis = 1, inplace = True)
df = df.iloc[ : , 0 : 2]
df

message    Label
0    Subject: re : 2 . 882 s - > np np\n\n> date : ...  notspam
1    Subject: s - > np + np\n\ndiscussion s - > np ...  notspam
2    Subject: 2 . 882 s - > np np\n\n. . . 's much ...  notspam
3    Subject: gent conference\n\n" listserv " inter...  notspam
4    Subject: query : causatives korean\n\nanyone p...  notspam
..                                                 ...      ...
206  Subject: `\n\nhello ! our naughty little city ...     spam
207  Subject: attract women ! ! ( 30740 )\n\nattrac...     spam
208  Subject: are serious . . . help retire 2 3 yea...     spam
209  Subject: " complete home business kit ! ! "\n\...     spam
210  Subject: advertisement\n\nmessage complies pro...     spam

[211 rows x 2 columns]

In [59]:
def text_preprocess(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()
    
    # Now just remove any stopwords and non alphabets
    nostop=[word for word in nopunc.split() if word.lower() not in stopwords.words('english') and word.isalpha()]
    
    return nostop

In [60]:
spam_messages = df[df["Label"] == "spam"]["message"]
notspam_messages = df[df["Label"] == "notspam"]["message"]
print(spam_messages)
print('------------------------------------------------------------------------')
print(notspam_messages)
print("No of spam messages : ",len(spam_messages))
print("No of notspam messages : ",len(notspam_messages))

193    Subject: great part-time summer job !\n\n* * *...
194    Subject: auto insurance rates too high ?\n\nde...
195    Subject: advertsing ? legal ! ! offer smtp ! !...
196    Subject: free trial membership\n\nlatest adult...
197    Subject: market millions , try free\n\nmessage...
198    Subject: lists software worldwide\n\norder for...
199    Subject: zero down internet opportunity !\n\n$...
200    Subject: re : free !\n\nhello , are offering f...
201    Subject: \n\ncomes porn , site does n't mess a...
202    Subject: even steal identity !\n\nare being in...
203    Subject: wanted ! home product assemblers !\n\...
204    Subject: secrets noveau rich\n\nstumbled upon ...
205    Subject: does debt-free sound ?\n\nsounds , he...
206    Subject: `\n\nhello ! our naughty little city ...
207    Subject: attract women ! ! ( 30740 )\n\nattrac...
208    Subject: are serious . . . help retire 2 3 yea...
209    Subject: " complete home business kit ! ! "\n\...
210    Subject: advertisement\n

In [61]:
# Download stopwords
# nltk.download('stopwords')

# Words in spam messages
spam_words = []
for each_message in spam_messages:
    spam_words += text_preprocess(each_message)
# print(spam_words)
print(f"Top 10 spam words are:\n {pd.Series(spam_words).value_counts().head(10)}")

Top 10 spam words are:
 report         77
email          49
money          42
information    38
offshore       38
mail           33
order          29
subject        27
sell           25
business       25
dtype: int64


In [62]:
df["message"] = df["message"].apply(text_preprocess)

df.head()


message    Label
0  [subject, np, np, date, sun, dec, est, michael...  notspam
1  [subject, np, np, discussion, np, np, reminds,...  notspam
2  [subject, np, np, much, restrictive, np, np, n...  notspam
3  [subject, gent, conference, listserv, internat...  notspam
4  [subject, query, causatives, korean, anyone, p...  notspam

In [63]:
# Convert messages (as lists of string tokens) to strings
df["message"] = df["message"].agg(lambda x: " ".join(map(str, x)))
df.head()


message    Label
0  subject np np date sun dec est michael mmorse ...  notspam
1  subject np np discussion np np reminds years a...  notspam
2  subject np np much restrictive np np np pro qu...  notspam
3  subject gent conference listserv international...  notspam
4  subject query causatives korean anyone point b...  notspam

In [64]:
vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(df["message"])
# print(bow_transformer)
# Fetch the vocabulary set
# print(f"20 BOW Features: {vectorizer.get_feature_names()[20:40]}")
print(f"Total number of vocab words: {len(vectorizer.vocabulary_)}")

Total number of vocab words: 10343


In [65]:
# Convert strings to vectors using BoW
messages_bow = bow_transformer.transform(df["message"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {messages_bow.shape}")
print(f"Amount of non-zero occurrences: {messages_bow.nnz}")

Shape of sparse matrix: (211, 10343)
Amount of non-zero occurrences: 26221


In [66]:
print(messages_bow)

  (0, 489)	1
  (0, 645)	1
  (0, 740)	2
  (0, 1271)	1
  (0, 1463)	1
  (0, 1938)	1
  (0, 1966)	2
  (0, 2273)	1
  (0, 2312)	1
  (0, 2640)	1
  (0, 2893)	1
  (0, 3040)	1
  (0, 3104)	1
  (0, 3337)	1
  (0, 3362)	1
  (0, 3536)	1
  (0, 3569)	1
  (0, 4507)	1
  (0, 4664)	1
  (0, 4769)	1
  (0, 4877)	3
  (0, 5136)	1
  (0, 5168)	1
  (0, 5273)	1
  (0, 5365)	1
  :	:
  (210, 7066)	1
  (210, 7319)	1
  (210, 7398)	1
  (210, 7669)	1
  (210, 7672)	1
  (210, 7832)	3
  (210, 7863)	1
  (210, 7893)	2
  (210, 7897)	1
  (210, 7951)	1
  (210, 8225)	1
  (210, 8283)	1
  (210, 8364)	1
  (210, 8399)	1
  (210, 8795)	1
  (210, 8797)	1
  (210, 8801)	1
  (210, 8851)	1
  (210, 8929)	2
  (210, 9452)	1
  (210, 9558)	1
  (210, 9603)	1
  (210, 9736)	1
  (210, 9816)	2
  (210, 10259)	1


In [67]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

# Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(211, 10343)


In [68]:
# Convert spam and ham labels to 0 and 1 (or, vice-versa)
FactorResult = pd.factorize(df["Label"])
df["Label"] = FactorResult[0]
df.head()

message  Label
0  subject np np date sun dec est michael mmorse ...      0
1  subject np np discussion np np reminds years a...      0
2  subject np np much restrictive np np np pro qu...      0
3  subject gent conference listserv international...      0
4  subject query causatives korean anyone point b...      0

In [69]:
# Split the dataset to train and test sets
msg_train, msg_test, label_train, label_test = train_test_split(
    messages_tfidf, df["Label"], test_size=0.2
)

print(f"train dataset features size: {msg_train.shape}")
print(f"train dataset label size: {label_train.shape}")

print(f"test dataset features size: {msg_test.shape}")
print(f"test dataset label size: {label_test.shape}")

train dataset features size: (168, 10343)
train dataset label size: (168,)
test dataset features size: (43, 10343)
test dataset label size: (43,)


In [70]:
print(messages_tfidf)


  (0, 10323)	0.11223050851993636
  (0, 10309)	0.10419553992331465
  (0, 10194)	0.11223050851993636
  (0, 9867)	0.25427630831033393
  (0, 9359)	0.07513762084665143
  (0, 9162)	0.11223050851993636
  (0, 9161)	0.3366915255598091
  (0, 9019)	0.07366904755364288
  (0, 9018)	0.11223050851993636
  (0, 9007)	0.09407267666521522
  (0, 8929)	0.03963334173986813
  (0, 8459)	0.11223050851993636
  (0, 8367)	0.14204579979039755
  (0, 8347)	0.07672380084015626
  (0, 8289)	0.059237784876972156
  (0, 8159)	0.06982152171804827
  (0, 7803)	0.0784480766598995
  (0, 7786)	0.06982152171804827
  (0, 7719)	0.2083910798466293
  (0, 7546)	0.08033680712363603
  (0, 6382)	0.36183868152694176
  (0, 6151)	0.06660093758205685
  (0, 6113)	0.11223050851993636
  (0, 6112)	0.1941366213549609
  (0, 6056)	0.05336678270845051
  :	:
  (210, 4026)	0.09777070946681612
  (210, 3472)	0.11158732928120356
  (210, 3414)	0.10730164574224677
  (210, 2930)	0.1682780024873506
  (210, 2927)	0.08282104231597218
  (210, 2298)	0.087384908

In [71]:
# Train an xgboost classifier
from xgboost import XGBClassifier

# Instantiate our model
clf = XGBClassifier()

# Fit the model to the training data
clf.fit(msg_train, label_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [72]:
# Make predictions
predict_train = clf.predict(msg_test)

print(
    f"Accuracy of Train dataset: {metrics.accuracy_score(label_test, predict_train):0.3f}"
)

Accuracy of Train dataset: 0.953


In [73]:
no_labels = np.arange(0, 78)
df_noLabel = []

for nl in no_labels : 
    df_unknown = pd.read_csv('TestData_nolabel/{}_unknown.txt'.format(nl))
    df_unknown.drop(df_unknown.columns[[0]], axis = 1, inplace = True)
    f = open('TestData_nolabel/{}_unknown.txt'.format(nl), "r")
    df_unknown['message'] = f.read()
    df_noLabel.append(df_unknown)
    

In [74]:
df_noLabel1 = pd.concat(df_noLabel).reset_index(drop = True)
df_noLabel1 = df_noLabel1.iloc[ : , 0 : 1]
df_noLabel1

message
0   Subject: base generated adjuncts\n\ndoes anyon...
1   Subject: basic journals\n\nare facing major cu...
2   Subject: query : tagalog philippine informants...
3   Subject: ancient vocal tract simulation\n\nwor...
4   Subject: re : 3 . 386 chomsky , mac concordanc...
..                                                ...
73  Subject: really cool hot videos !\n\nattention...
74  Subject: improvements wound care !\n\nhello , ...
75  Subject: \n\nb r g n r f r e s 1 - stop travel...
76  Subject: credit program " guaranteed credit "\...
77  Subject: free promotional offer\n\n' ' own 100...

[78 rows x 1 columns]

In [75]:
df_noLabel1["message"] = df_noLabel1["message"].apply(text_preprocess)
df_noLabel1.head()

message
0  [subject, base, generated, adjuncts, anyone, r...
1  [subject, basic, journals, facing, major, cuts...
2  [subject, query, tagalog, philippine, informan...
3  [subject, ancient, vocal, tract, simulation, w...
4  [subject, chomsky, mac, concordance, seems, wh...

In [76]:
# Convert messages (as lists of string tokens) to strings
df_noLabel1["message"] = df_noLabel1["message"].agg(lambda x: " ".join(map(str, x)))
df_noLabel1.head()
df_noLabel1
# clf.predict(df_noLabel1)


message
0   subject base generated adjuncts anyone referen...
1   subject basic journals facing major cuts libra...
2   subject query tagalog philippine informants na...
3   subject ancient vocal tract simulation work fo...
4   subject chomsky mac concordance seems whatever...
..                                                ...
73  subject really cool hot videos attention warni...
74  subject improvements wound care hello name kev...
75  subject b r g n r f r e stop travel supplier a...
76  subject credit program guaranteed credit bad c...
77  subject free promotional offer free web site s...

[78 rows x 1 columns]

In [77]:
# vectorizer1 = CountVectorizer()
# # bow_transformer = vectorizer1.fit(df_noLabel1["message"])

# # Fetch the vocabulary set
# print(f"20 BOW Features: {vectorizer1.get_feature_names()[20:40]}")
# print(f"Total number of vocab words: {len(vectorizer1.vocabulary_)}")

In [78]:
# Convert strings to vectors using BoW
messages_bow_NL = bow_transformer.transform(df_noLabel1["message"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {messages_bow_NL.shape}")
print(f"Amount of non-zero occurrences: {messages_bow_NL.nnz}")

Shape of sparse matrix: (78, 10343)
Amount of non-zero occurrences: 7822


In [79]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow_NL)

# Transform entire BoW into tf-idf corpus
messages_tfidf_NL = tfidf_transformer.transform(messages_bow_NL)
print(messages_tfidf_NL.shape)

(78, 10343)


In [80]:
from sklearn.metrics import classification_report, f1_score, ConfusionMatrixDisplay, confusion_matrix

In [81]:
model_pred = clf.predict(messages_tfidf_NL)
model_pred = pd.DataFrame(model_pred)
model_pred = model_pred.replace({0: 'nospam', 1: 'spam'})
model_pred = model_pred.to_csv('ANSWER.csv')
